In [5]:
from utils.load_env import load_env

envs = load_env()

# ComfyUI 服务地址
COMFYUI_BASE_URL = envs.get("COMFYUI_URL")

print(f"Using ComfyUI URL: {COMFYUI_BASE_URL}")

Environment variables loaded.
Using ComfyUI URL: http://home.chrissong.top:3818


In [11]:
from utils.load_json import load_json

# WORKFLOW_API_FILE = "comfyui_wf_api/base_api.json"
WORKFLOW_API_FILE = "comfyui_wf_api/image2image.json"

prompt_json = load_json(WORKFLOW_API_FILE)

print(prompt_json)

{'3': {'inputs': {'seed': 1111537140283334, 'steps': 20, 'cfg': 8, 'sampler_name': 'dpmpp_2m', 'scheduler': 'normal', 'denoise': 0.8700000000000001, 'model': ['14', 0], 'positive': ['6', 0], 'negative': ['7', 0], 'latent_image': ['12', 0]}, 'class_type': 'KSampler', '_meta': {'title': 'K采样器'}}, '6': {'inputs': {'text': 'photograph of victorian woman with wings, sky clouds, meadow grass\n', 'clip': ['14', 1]}, 'class_type': 'CLIPTextEncode', '_meta': {'title': 'CLIP文本编码'}}, '7': {'inputs': {'text': 'watermark, text\n', 'clip': ['14', 1]}, 'class_type': 'CLIPTextEncode', '_meta': {'title': 'CLIP文本编码'}}, '8': {'inputs': {'samples': ['3', 0], 'vae': ['14', 2]}, 'class_type': 'VAEDecode', '_meta': {'title': 'VAE解码'}}, '9': {'inputs': {'filename_prefix': 'ComfyUI', 'images': ['8', 0]}, 'class_type': 'SaveImage', '_meta': {'title': '保存图像'}}, '10': {'inputs': {'image': 'example.png'}, 'class_type': 'LoadImage', '_meta': {'title': '加载图像'}}, '12': {'inputs': {'pixels': ['18', 0], 'vae': ['14', 2

需要使用异步的 websocket 链接 才能成功运行

In [ ]:
import websockets
import json
import uuid
import urllib.request
from urllib.parse import urlparse
from typing import Dict, Any

client_id = str("bd3fc1077c9845508e26b06ef2dbeff1")


def queue_prompt(prompt, prompt_id):
    p = {"prompt": prompt, "client_id": client_id, "prompt_id": prompt_id}
    data = json.dumps(p).encode("utf-8")
    req = urllib.request.Request("{}/prompt".format(COMFYUI_BASE_URL), data=data)
    urllib.request.urlopen(req).read()


async def get_history(prompt_id):
    with urllib.request.urlopen(
        "{}/history/{}".format(COMFYUI_BASE_URL, prompt_id)
    ) as response:
        return json.loads(response.read())


async def send_prompt_and_listen(prompt: Dict[str, Any]):
    # 生成正确的WebSocket地址
    parsed_url = urlparse(COMFYUI_BASE_URL)
    ws_scheme = "wss" if parsed_url.scheme == "https" else "ws"
    ws_url = f"{ws_scheme}://{parsed_url.netloc}/ws?clientId={client_id}"
    print(f"尝试连接WebSocket: {ws_url}")

    try:
        async with websockets.connect(
            ws_url, extra_headers={"Origin": COMFYUI_BASE_URL}
        ) as websocket:
            print("✅ WebSocket连接成功！")

            prompt_id = str(uuid.uuid4())
            queue_prompt(prompt, prompt_id)
            print("📤 已发送prompt")

            # 2. 持续监听返回信息
            while True:
                response = await websocket.recv()
                response_data = json.loads(response)

                # 根据消息类型处理不同响应
                if response_data.get("type") == "status":
                    # 处理状态更新（如队列位置、正在处理等）
                    status = response_data.get("status", {})
                    print(
                        f"📊 状态更新: 队列位置 {status.get('queue_position')}，状态: {status.get('status')}"
                    )

                elif response_data.get("type") == "executing":
                    # 处理正在执行的节点信息
                    node = response_data.get("node")
                    print(f"▶️ 正在执行节点: {node}")

                elif response_data.get("type") == "executed":
                    # 处理节点执行完成信息
                    node = response_data['data'].get("node")
                    output = response_data['data'].get("output")
                    print(f"✅ 节点执行完成: {node}")
                    # 如果有图像输出，提取图像信息
                    if "images" in output:
                        print(f"🖼️ 生成图像数量: {len(output['images'])}")
                        for img in output["images"]:
                            print(
                                f"   - 图像名称: {img['filename']}, 类型: {img['type']}"
                            )

                elif response_data.get("type") == "error":
                    # 处理错误信息
                    error = response_data.get("message")
                    print(f"❌ 错误: {error}")
                    break  # 发生错误时退出监听

                elif response_data.get("type") == "done":
                    # 处理所有任务完成信息
                    print("🏁 所有任务执行完成")
                    break  # 完成后退出监听

    except websockets.exceptions.InvalidStatusCode as e:
        print(f"❌ 握手失败，状态码: {e.status_code}")
        if e.status_code == 400:
            print("- 请检查路径是否正确(/ws)、协议是否匹配或跨域设置")
    except Exception as e:
        print(f"❌ 连接错误: {str(e)}")


if __name__ == "__main__":
    # 启动事件循环
    await send_prompt_and_listen(prompt_json)